In [2]:
from scipy.io import loadmat
from PIL import Image, ImageDraw
from IPython.display import Image as IPImage
# import os

import numpy as np
import pandas as pd
import os

# from PIL import Image

import torch

from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
import torchvision.transforms.functional  as TF

# from utils import img_utils
import matplotlib.pyplot as plt

In [3]:
CUR_DIR = os.path.join(os.path.dirname(os.path.abspath("__file__")))
CUR_DIR

'/Users/tonmoy/Library/CloudStorage/OneDrive-IndianaUniversity/Research/Education Project/Gaze/src/gazepoint/gfie/test'

In [4]:
WORKSPACE_DIR = os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(CUR_DIR)))))
DATASET_DIR = os.path.join(WORKSPACE_DIR, "Data", "gaze360")
DATASET_DIR

'/Users/tonmoy/Library/CloudStorage/OneDrive-IndianaUniversity/Research/Education Project/Data/gaze360'

In [5]:
mat = loadmat(os.path.join(CUR_DIR, "metadata.mat"))
mat.keys()

dict_keys(['__header__', '__version__', '__globals__', 'recordings', 'recording', 'frame', 'ts', 'target_cam', 'target_pos3d', 'target_pos2d', 'person_identity', 'person_cam', 'person_eyes3d', 'person_eyes2d', 'person_body_bbox', 'person_head_bbox', 'person_face_bbox', 'person_eye_left_bbox', 'person_eye_right_bbox', 'gaze_dir', 'splits', 'split'])

In [6]:
mat["recordings"]

array([[array(['rec_000'], dtype='<U7'), array(['rec_001'], dtype='<U7'),
        array(['rec_002'], dtype='<U7'), array(['rec_003'], dtype='<U7'),
        array(['rec_004'], dtype='<U7'), array(['rec_005'], dtype='<U7'),
        array(['rec_006'], dtype='<U7'), array(['rec_007'], dtype='<U7'),
        array(['rec_008'], dtype='<U7'), array(['rec_009'], dtype='<U7'),
        array(['rec_010'], dtype='<U7'), array(['rec_011'], dtype='<U7'),
        array(['rec_012'], dtype='<U7'), array(['rec_013'], dtype='<U7'),
        array(['rec_014'], dtype='<U7'), array(['rec_015'], dtype='<U7'),
        array(['rec_016'], dtype='<U7'), array(['rec_017'], dtype='<U7'),
        array(['rec_018'], dtype='<U7'), array(['rec_019'], dtype='<U7'),
        array(['rec_020'], dtype='<U7'), array(['rec_021'], dtype='<U7'),
        array(['rec_022'], dtype='<U7'), array(['rec_023'], dtype='<U7'),
        array(['rec_024'], dtype='<U7'), array(['rec_025'], dtype='<U7'),
        array(['rec_026'], dtype='<U7'

In [7]:
mat["recording"][0]

array([ 0,  0,  0, ..., 79, 79, 79], dtype=int32)

In [8]:
mat["frame"]

array([[  0,   1,   2, ..., 521, 522, 522]])

In [9]:
mat["ts"]

array([[ 0.        ,  0.12525296,  0.25050497, ..., 63.37850809,
        63.50351596, 63.50351596]])

In [10]:
mat["person_identity"][0][0]

15

In [11]:
mat["target_pos3d"]

array([[ 1.69389339,  1.75942589, -0.62393989],
       [ 1.68793889,  1.76528217, -0.62475749],
       [ 1.68868502,  1.76575734, -0.62507184],
       ...,
       [ 0.67011673, -0.48928031, -0.51240202],
       [ 0.64817876, -0.47156056, -0.44699191],
       [ 0.64817876, -0.47156056, -0.44699191]])

In [12]:
mat["person_head_bbox"]

array([[0.58718794, 0.4859066 , 0.04633445, 0.0382576 ],
       [0.5837803 , 0.48372877, 0.04876879, 0.04026759],
       [0.5783111 , 0.47928429, 0.05532767, 0.04568315],
       ...,
       [0.51265723, 0.47321415, 0.08451395, 0.06978178],
       [0.49715102, 0.47933304, 0.08145172, 0.06725335],
       [0.51202176, 0.47195637, 0.08704371, 0.07187057]])

In [59]:
def prepare_dataset(mat, dstype):
    orig_x, orig_y = 3382, 4096
    frame = mat["frame"]
    # person_eyes_2d = mat["person_eyes2d"]
    # person_eyes_3d = mat["person_eyes3d"]

    himg = list()
    simg = list()
    frame_id = list()
    hbbox = list()
    fbbox = list()
    bbbox = list()
    person_eyes2d = list()
    person_eyes3d = list()
    gaze_direction = list()
    target2d = list()
    target3d = list()


    for i in range(len(frame[0])):
        if  mat["split"][0][i] == dstype:
            himg += [os.path.join(
                mat["recordings"][0][mat["recording"][0][i]].item(),
                "head",
                "%06d" % mat["person_identity"][0][i].item(),
                "%06d.jpg" % mat["frame"][0][i].item(),
            )]
            simg += [os.path.join(
                mat["recordings"][0][mat["recording"][0][i]].item(),
                "body",
                "%06d" % mat["person_identity"][0][i].item(),
                "%06d.jpg" % mat["frame"][0][i].item(),
            )]
            frame_id += [mat["frame"][0][i]]
            hbbox += [mat["person_head_bbox"][i]]
            # fbbox += [mat["person_face_bbox"][i]]
            # bbbox += [mat["person_body_bbox"][i]]
            person_eyes2d += [mat["person_eyes2d"][i]]
            person_eyes3d += [mat["person_eyes3d"][i]]
            gaze_direction += [mat["gaze_dir"][i]]
            target2d += [mat["target_pos2d"][i]]
            target3d += [mat["target_pos3d"][i]]


    df = pd.DataFrame({
        "simg": simg,
        "himg": himg,
        "frame_id": frame_id,
        "h_x_min" : [x[0] for x in hbbox],
        "h_y_min" : [x[1] for x in hbbox],
        "h_x_max" : [x[2] for x in hbbox],
        "h_y_max" : [x[3] for x in hbbox],
        "eye_u" : [x[0] for x in person_eyes2d],
        "eye_v" : [x[1] for x in person_eyes2d],
        "eye_X" : [x[0] for x in person_eyes3d],
        "eye_Y" : [x[1] for x in person_eyes3d],
        "eye_Z" : [x[2] for x in person_eyes3d],
        "gaze_dirX": [x[0] for x in gaze_direction],
        "gaze_dirY": [x[1] for x in gaze_direction],
        "gaze_dirZ": [x[2] for x in gaze_direction],
        "gaze_u" : [x[0] for x in target2d],
        "gaze_v": [x[1] for x in target2d],
        "gaze_X": [x[0] for x in target3d],
        "gaze_Y": [x[1] for x in target3d],
        "gaze_Z": [x[2] for x in target3d],
    })

    return df



In [71]:
## Prepare Train Dataset
df = prepare_dataset(mat, 0)
df.to_csv(os.path.join(DATASET_DIR, "train.csv"))
df.shape

(126928, 20)

In [72]:
## Prepare Validation Dataset
df = prepare_dataset(mat, 1)
df.to_csv(os.path.join(DATASET_DIR, "validation.csv"))
df.shape

(17038, 20)

In [73]:
## Prepare Test Dataset
df = prepare_dataset(mat, 2)
df.to_csv(os.path.join(DATASET_DIR, "test.csv"))
df.shape

(25969, 20)

In [13]:
# img_path = os.path.join(DATASET_DIR, "imgs", mat["recordings"]) # , "body", '%06d' % mat["person_identity"][0], "%06d.jpg" % mat["frame"][0]
# img_path
orig_x, orig_y = 3382, 4096
recordings = mat["recordings"]
recording = mat["recording"]
splits = mat["splits"]
split = mat["split"]
person_head_bbox = mat["person_head_bbox"]
person_face_bbox = mat["person_face_bbox"]
person_body_bbox = mat["person_body_bbox"]
person_identity = mat["person_identity"]
gaze_dir = mat["gaze_dir"]
frame = mat["frame"]
person_eyes_2d = mat["person_eyes_2d"]
person_eyes_3d = mat["person_eyes_3d"]

i = 7370
img_path = os.path.join(
    DATASET_DIR,
    "imgs",
    recordings[0][recording[0][i]].item(),
    "head",
    "%06d" % mat["person_identity"][0][i].item(),
    "%06d.jpg" % mat["frame"][0][i].item(),
)
# img_path
# img = Image.open(img_path)
# Image._show(img)

In [80]:
from transformers import pipeline

checkpoint = "vinvino02/glpn-nyu"
depth_estimator = pipeline("depth-estimation", model=checkpoint, device=0)

sample_img = Image.open(os.path.join(DATASET_DIR, "imgs", lines[0].split(" ")[0]))
preds = depth_estimator(sample_img)

In [82]:
sample_img.size

(217, 217)

In [85]:
preds

{'predicted_depth': tensor([[[5.3033, 5.9054, 5.7112,  ..., 4.9678, 5.0379, 5.1203],
          [6.2931, 7.2684, 7.4032,  ..., 4.8657, 5.0426, 4.9449],
          [6.5597, 7.6745, 7.8852,  ..., 5.2611, 5.4131, 5.0675],
          ...,
          [3.2218, 3.0695, 2.8769,  ..., 5.6232, 5.4847, 5.1200],
          [3.4083, 3.1831, 2.9930,  ..., 5.5539, 5.3422, 5.0289],
          [3.8051, 3.6194, 3.3994,  ..., 5.2701, 5.1984, 4.9153]]]),
 'depth': <PIL.Image.Image image mode=L size=217x217>}

In [86]:
preds["predicted_depth"].shape, preds["depth"].size

(torch.Size([1, 192, 192]), (217, 217))

In [89]:
import numpy as np


cam_kinect = np.array([910.78759766, 910.21258545, 961.65966797, 554.11016846])
np.save(os.path.join(DATASET_DIR, "CameraKinect.npy"), cam_kinect)